In [1]:
from PIL import Image
import os, glob, numpy as np
from sklearn.model_selection import train_test_split

In [30]:
classes = ['갈비구이','갈비탕','갈치구이','감자채볶음','계란찜','김밥','김치찌개','깻잎장아찌']
nb_classes = len(classes)

In [3]:
import os, glob, numpy as np
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout
from keras.callbacks import EarlyStopping, ModelCheckpoint
import matplotlib.pyplot as plt
import keras.backend.tensorflow_backend as K
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

Using TensorFlow backend.


In [4]:
batch_size = 16
# 학습 이미지에 적용한 augmentation 인자를 지정
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

# 검증 및 테스트 이미지는 augmentation을 적용하지 않음. 모델 성능을 평가할 때에는 이미지 원본을 사용
validation_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

In [5]:
imgdir = 'food_img/'

# 이미지를 배치 단위로 불러와 줄 generator
train_generator = train_datagen.flow_from_directory(
        imgdir + 'train',  # this is the target directory
        target_size=(150, 150),  # 모든 이미지의 크기가 150x150로 조정
        batch_size=batch_size,
        class_mode='categorical')  # categorical_crossentropy 손실 함수를 사용하므로 categorical형태로 라벨을 불러와야 합니다.

validation_generator = validation_datagen.flow_from_directory(
        imgdir + 'test',
        target_size=(150, 150),
        batch_size=batch_size,
        class_mode='categorical')

test_generator = test_datagen.flow_from_directory(
        imgdir + 'test',
        target_size=(150, 150),
        batch_size=batch_size,
        class_mode='categorical')

Found 1920 images belonging to 8 classes.
Found 480 images belonging to 8 classes.
Found 480 images belonging to 8 classes.


In [6]:
# with K.tf_ops.device('/device:GPU:0'):
model = Sequential()
model.add(Conv2D(32, (3,3), padding="same", input_shape=(150,150,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3,3), padding="same", activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(nb_classes, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [7]:
from keras.models import model_from_json
model.load_weights('0.hdf5')

In [8]:
finalAcc = model.evaluate(validation_generator)[1]
print('\nAccuracy: {:.4f}'.format(finalAcc))

C:\ProgramData\Anaconda3\envs\pytorch\lib\site-packages\PIL\Image.py:993: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "


30/30 [==============================] - 6s 185ms/step

Accuracy: 0.7896


In [62]:
import glob
import PIL.Image as pilimg
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt


In [63]:
img = Image.open('김치찌개.jpg')
img = img.convert("RGB")
img = img.resize((150, 150))
data = np.asarray(img)
data = data.reshape(1,150,150,3)

In [64]:
cate = model.predict(data)

In [65]:
for i, num in enumerate(cate[0]):
    if int(num) == 1:
        answer = i
        break

In [66]:
answer = classes[i]
answer

'김치찌개'

In [67]:
import pandas as pd

In [79]:
df = pd.read_excel('찌개.xlsx')
df = df[df['Unnamed: 0'] == answer][['단백질(g)', '총 당류(g)','탄수화물(g)','나트륨(mg)']]
df

,단백질(g),총 당류(g),탄수화물(g),나트륨(mg)
17,11.3,1.3,6.1,488.1


In [80]:
df = df.reset_index()
df

,index,단백질(g),총 당류(g),탄수화물(g),나트륨(mg)
0,17,11.3,1.3,6.1,488.1


In [81]:
df['index'] = answer
df

,index,단백질(g),총 당류(g),탄수화물(g),나트륨(mg)
0,김치찌개,11.3,1.3,6.1,488.1


In [82]:
df.to_excel('dailyInfo.xlsx')